In [1]:
import re
from collections import Counter
import spacy
word2vec = spacy.load("en_core_web_sm")
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from random import randrange


In [2]:
#replace spacy.load("en") with spacy.load("en_core_web_sm")
#nltk.download("stopwords")
#nltk.download('punkt')
#python -m spacy download en   ...do this in the conda terminal

In [3]:

    import requests

    url = "https://api-football-beta.p.rapidapi.com/players/topscorers"

    querystring = {"season":"2022","league":"39"}

    headers = {
        "X-RapidAPI-Key": "62a02ca5ccmsha68a5acf6c698d9p1ffc12jsnbe2b9a765139",
        "X-RapidAPI-Host": "api-football-beta.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    #print (type(response.json()))
    #print(response.json())
    df = response.json()

In [17]:
def askplayer(name):
        askq = input("I know a lot about football, if you want I can tell you about some of the top scorers in the premiere league. So what do you say, Yes or No? ")
        if askq =='yes' or askq == 'Yes':
            print("Here are the names of some of the top scorers I know, Choose any of those names and I'll tell you all about them")
            print("E. Haaland, H. Kane, A. Mitrović, I. Toney, L. Trossard, M. Almirón, P. Foden, Roberto Firmino")  
           #playerchoose = input("So what's your choice ")
            while True:
                playerchoose = input("So what's your choice ")
                if playerchoose in playercode.keys():
                    playerid = (playercode[playerchoose])
                    #print(playerid)
                    print("Ah yes... " + playerchoose + "I know all about him. Did you know in this season he has scored a total of " 
                          + str(goals) + " goals and he's attempted a grande total of " + str(shot) +" shots")
                    break




                else:
                    print("That's not one of the names I gave you... Try again.")
                    

In [75]:
playerid = 0
goals = df['response'][playerid]['statistics'][0]['goals']['total']
shot = df['response'][playerid]['statistics'][0]['shots']['total']
passing = df['response'][playerid]['statistics'][0]['passes']['total']
playercode = {'E. Haaland':0, 'H. Kane':1, 'A. Mitrović':2, 'I. Toney':3, 'L. Trossard':4, 'M. Almirón':5, 'P. Foden':6, 'Roberto Firmino':7}

In [76]:
#askplayer("s")
#askp = askplayer("s")

In [77]:
response_a = "I've been watching football since I was two"
response_b = "I hate Manchester City"
response_c = "yeah I like {}"
response_d = "my favorite team is Manchester United"
response_e = "I've been watching football since I was two"
blank_spot = "player"


responses = [response_a, response_b, response_c, response_d, response_e]

In [78]:
exit_commands = ("quit", "goodbye", "exit", "no","bye")

In [79]:
def preprocess(input_sentence):
    input_sentence = input_sentence.lower()
    input_sentence = re.sub(r'[^\w\s]','',input_sentence)
    tokens = word_tokenize(input_sentence)
    input_sentence = [i for i in tokens if not i in stop_words]
    return(input_sentence)

def compare_overlap(user_message, possible_response):
    similar_words = 0
    for token in user_message:
        if token in possible_response:
              similar_words += 1
    return similar_words
  
def extract_nouns(tagged_message):
    message_nouns = list()
    for token in tagged_message:
        if token[1].startswith("N"):
            message_nouns.append(token[0])
    return message_nouns

def compute_similarity(tokens, category):
    output_list = list()
    for token in tokens:
        output_list.append([token.text, category.text, token.similarity(category)])
    return output_list
  


In [90]:
class ChatBot:

    def make_exit(self, user_message):
        for exit_command in exit_commands:
            if exit_command in user_message:
                print("Ok, bye!")
                return True
            

    def askplayer(name):
        askq = input("I know a lot about football, if you want I can tell you about some of the top scorers in the premiere league. So what do you say, Yes or No? ")
        if askq =='yes' or askq == 'Yes':
            print("Here are the names of some of the top scorers I know, Choose any of those names and I'll tell you all about them")
            print("E. Haaland, H. Kane, A. Mitrović, I. Toney, L. Trossard, M. Almirón, P. Foden, Roberto Firmino")  
           #playerchoose = input("So what's your choice ")
            while True:
                playerchoose = input("So what's your choice ")
                if playerchoose in playercode.keys():
                    playerid = (playercode[playerchoose])
                    #print(playerid)
                    print("Ah yes... " + playerchoose + "I know all about him. Did you know in this season he has scored a total of " 
                          + str(goals) + " goals and he's attempted a grande total of " + str(shot) +" shots")
                    
                    #user_message == input("What else do you want to ask me?")
                if playerchoose == "bye":
                    break
                    cantina_bot.chat()

                else:
                    print("That's not one of the names I gave you... Try again.")
                    
        
    def chat(self):
        user_message = input("Hey! I know a lot about football is there anything you want to know? ")
        if "top scorer" in user_message:
            askplayer("name")

    
        while not self.make_exit(user_message):
            user_message = self.respond(user_message)
        

    
    def find_intent_match(self, responses, user_message):
        bow_user_message = Counter(preprocess(user_message))
        processed_responses = [Counter(preprocess(response)) for response in responses]
        similarity_list = [compare_overlap(doc, bow_user_message) for doc in processed_responses]
        response_index = similarity_list.index(max(similarity_list))
        return responses[response_index]


    def respond(self, user_message):
        best_response = self.find_intent_match(responses, user_message)
        entity = self.find_entities(user_message)
        print(best_response.format(entity))
        input_message = input("Do you have anything else that you want to talk about? ")           
        return input_message
        
    def find_entities(self, user_message):
          # extract candidate entities
        tagged_user_message = pos_tag(preprocess(user_message))
        message_nouns = extract_nouns(tagged_user_message)
        tokens = word2vec(" ".join(message_nouns))
        category = word2vec(blank_spot)
        word2vec_result = compute_similarity(tokens, category)
        word2vec_result.sort(key=lambda x: x[2])
        if len(word2vec_result) < 1:
            return blank_spot
        else:
            return word2vec_result[-1][0]


        

cantina_bot = ChatBot()
cantina_bot.chat()

Hey! I know a lot about football is there anything you want to know? top scorer
I know a lot about football, if you want I can tell you about some of the top scorers in the premiere league. So what do you say, Yes or No? yes
Here are the names of some of the top scorers I know, Choose any of those names and I'll tell you all about them
E. Haaland, H. Kane, A. Mitrović, I. Toney, L. Trossard, M. Almirón, P. Foden, Roberto Firmino
So what's your choice E. Haaland


C:\Users\Dell\AppData\Local\Temp\ipykernel_10856\1344654111.py:25: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  output_list.append([token.text, category.text, token.similarity(category)])


Ah yes... E. HaalandI know all about him. Did you know in this season he has scored a total of 18 goals and he's attempted a grande total of 43 shots
I've been watching football since I was two
Do you have any other questions? yes
I've been watching football since I was two
Do you have any other questions? manchester city
I hate Manchester City
Do you have any other questions? man united
my favorite team is Manchester United
Do you have any other questions? no
Ok, bye!


In [45]:
user_message = input("Hey! I know a lot about football is there anything you want to know? ")

Hey! I know a lot about football is there anything you want to know? I know everything


In [98]:
print(randrange(4))

2


In [99]:
import  pandas as pd

In [113]:
d1 = pd.read_csv("111.csv")
d1.head()

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,11/08/2019,16:30,Matchweek 1,Sun,Home,W,4,0,Chelsea,2.2,1.0,46,"73,620",David de Gea,4-2-3-1,Anthony Taylor,Match Report,NaN
1,19/08/2019,20:00,Matchweek 2,Mon,Away,D,1,1,Wolves,1.2,0.3,65,"31,314",David de Gea,4-2-3-1,Jonathan Moss,Match Report,NaN
2,24/08/2019,15:00,Matchweek 3,Sat,Home,L,1,2,Crystal Palace,2.5,1.1,71,"73,454",David de Gea,4-2-3-1,Paul Tierney,Match Report,NaN
3,31/08/2019,12:30,Matchweek 4,Sat,Away,D,1,1,Southampton,1.5,0.9,59,"30,499",Ashley Young,4-2-3-1,Mike Dean,Match Report,NaN
4,14/09/2019,15:00,Matchweek 5,Sat,Home,W,1,0,Leicester City,1.2,0.6,42,"73,689",Ashley Young,4-2-3-1,Martin Atkinson,Match Report,NaN


In [121]:
d1['Result']

0     W
1     D
2     L
3     D
4     W
5     L
6     D
7     L
8     D
9     W
10    L
11    W
12    D
13    D
14    W
15    W
16    D
17    L
18    W
19    W
20    L
21    W
22    L
23    L
24    D
25    W
26    W
27    D
28    W
29    D
30    W
31    W
32    W
33    W
34    D
35    W
36    D
37    W
Name: Result, dtype: object